In [ ]:
import nltk
nltk.download(['stopwords','punkt_tab'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip uninstall -y numpy scipy
!pip install --no-cache-dir numpy gensim scipy

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 142.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 193.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 204.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 69.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you 

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 10.1 MB/s eta 0:00:00


### Approach 1 - Count Vectorizer

In [ ]:
import pandas as pd
import numpy as np
import re
import emoji

import xgboost as xgb

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB


from nltk.corpus import stopwords

In [ ]:
kaggle_review = pd.read_csv('/content/Reviews.csv')
lb = LabelBinarizer()
kaggle_review['rating'] = lb.fit_transform(kaggle_review['sentiment'])
kaggle_review.drop(columns=['sentiment'],inplace=True)
kaggle_review.rename(columns={'review':'Review'},inplace=True)

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopword(text):
    return " ".join([word for word in text.split() if word not in stop_words])

In [ ]:
# @title
contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
    "'ve": " have",
    "'ll": " will"
    }

def decontract(text):
  decontracted = []
  for word in text.split():
    if word in contractions:
      word = contractions[word]
    decontracted.append(word)
  return ' '.join(decontracted)

In [ ]:
def cleaning(text):
  text = text.lower()             # lower casing
  text = re.sub('<.*?>','',text)   #remove HTML Tags
  text = emoji.demojize(text)         # remove emojis
  text = re.sub('https?://\S*|www\.\S*','',text)   # remove URL's
  text = decontract(text)               # decontraction
  text = re.sub(r"'\w+", "", text)        # remove any letter after '
  text = re.sub(r"\d+", " ", text)        # remove digits [0-9]
  text = re.sub(r'\b\*\b', '', re.sub(r'(\w)\*(\w)', r'\1\2', text))   # remove * unless its inbetween letters
  text = re.sub(r'[^\x00-\x7F]+', '', text)     # remove ASCII
  text = re.sub(r'[^a-z\s]', ' ', text)         # remove punctuations
  text = re.sub(r'\s+', ' ', text).strip()      # remove space
  return text

kaggle_review['Review'] = kaggle_review['Review'].apply(cleaning)
kaggle_review['Review'] = kaggle_review['Review'].apply(remove_stopword)

In [ ]:
kaggle_review_train = kaggle_review[:20000]
kaggle_review_test = kaggle_review[30000:40000]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(kaggle_review_train['Review'],kaggle_review_train['rating'],test_size=0.2,random_state=42)
x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape

((16000,), (4000,), (16000,), (4000,))

In [ ]:
cv = CountVectorizer()
x_train_bow = cv.fit_transform(x_train).toarray()
x_test_bow = cv.transform(x_test).toarray()

x_train_bow.shape , x_test_bow.shape

((16000, 63298), (4000, 63298))

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train_bow,y_train)
y_pred = rfc.predict(x_test_bow)
print('accuray - ',accuracy_score(y_test,y_pred))


accuray -  0.84975


In [ ]:
test = cv.transform(kaggle_review_test['Review']).toarray()
test.shape

(10000, 63298)

In [ ]:
y_pred = rfc.predict(test)
y_testt = kaggle_review_test['rating']
print('accuray - ',accuracy_score(y_testt,y_pred))


accuray -  0.8434


In [ ]:
scrape_review = pd.read_csv('/content/F_review.csv')[['rating','summary','review']]
scrape_review['Review'] = scrape_review['summary'] +" "+ scrape_review['review']
scrape_review.drop(columns=['summary','review'],inplace=True)

In [ ]:
test_review = scrape_review.sample(10000,random_state=4)
test_review['Review'] = test_review['Review'].apply(cleaning)
test_review['Review'] = test_review['Review'].apply(remove_stopword)

testing_rev = cv.transform(test_review['Review']).toarray()

In [ ]:
y_pred = rfc.predict(testing_rev)
y_testt = test_review['rating']
print('accuray - ',accuracy_score(y_testt,y_pred))

accuray -  0.8861


In [ ]:
text = 'Overall the experience was so crap. but in some portion the movie was bad'

def sentiment(text):
  text = cleaning(text)
  text = remove_stopword(text)
  input = cv.transform([text]).toarray()
  pred = rfc.predict(input)
  prediction = 'positive' if pred[0]==1 else 'negative'
  return prediction

sentiment(text)

'negative'

### Approach 2 - keras tokenizer and keras embedding

In [ ]:
import os
import json
import pandas as pd
import re
import emoji
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Embedding,LSTM,Input, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
kaggle_review = pd.read_csv('/content/Reviews.csv')
kaggle_review.shape

(50000, 2)

In [ ]:
lb = LabelBinarizer()
kaggle_review['rating'] = lb.fit_transform(kaggle_review['sentiment'])
kaggle_review.drop(columns=['sentiment'],inplace=True)
kaggle_review.rename(columns={'review':'Review'},inplace=True)

In [ ]:
# @title
contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
    "'ve": " have",
    "'ll": " will"
    }

def decontract(text):
  decontracted = []
  for word in text.split():
    if word in contractions:
      word = contractions[word]
    decontracted.append(word)
  return ' '.join(decontracted)

In [ ]:
def cleaning(text):
  text = text.lower()             # lower casing
  text = re.sub('<.*?>','',text)   #remove HTML Tags
  text = emoji.demojize(text)         # remove emojis
  text = re.sub('https?://\S*|www\.\S*','',text)   # remove URL's
  text = decontract(text)               # decontraction
  text = re.sub(r"'\w+", "", text)        # remove any letter after '
  text = re.sub(r"\d+", " ", text)        # remove digits [0-9]
  text = re.sub(r'\b\*\b', '', re.sub(r'(\w)\*(\w)', r'\1\2', text))   # remove * unless its inbetween letters
  text = re.sub(r'[^\x00-\x7F]+', '', text)     # remove ASCII
  text = re.sub(r'[^a-z\s]', ' ', text)         # remove punctuations
  text = re.sub(r'\s+', ' ', text).strip()      # remove space
  return text

kaggle_review['Review'] = kaggle_review['Review'].apply(cleaning)

In [ ]:
train_data,test_data = train_test_split(kaggle_review,test_size=0.2,random_state=41)
train_data.shape ,test_data.shape

((40000, 2), (10000, 2))

In [ ]:
train_reviews = train_data['Review'].astype(str).tolist()
test_reviews = test_data['Review'].astype(str).tolist()

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_reviews)

In [ ]:
train_sequences = pad_sequences(tokenizer.texts_to_sequences(train_reviews),maxlen=200)
test_sequences = pad_sequences(tokenizer.texts_to_sequences(test_reviews),maxlen=200)

In [ ]:
y_train = train_data['rating']
y_test = test_data['rating']

train_sequences.shape,test_sequences.shape,y_train.shape,y_test.shape

((40000, 200), (10000, 200), (40000,), (10000,))

In [ ]:
# model = Sequential()
# model.add(Embedding(input_dim=5000,output_dim=100))
# model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.3,return_sequences=False))
# model.add(Dense(1,activation='sigmoid'))


# model = Sequential()
# model.add(Embedding(input_dim=5000, output_dim=100, input_length=200))
# model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.3))
# model.add(Dense(1, activation='sigmoid'))



input_layer = Input(shape=(200,))  # ✅ No batch shape, just the input shape
x = Embedding(input_dim=5000, output_dim=100)(input_layer)
x = LSTM(128, dropout=0.2, recurrent_dropout=0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output)

In [ ]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [ ]:
model.fit(train_sequences,y_train,epochs=8,validation_split=0.2,batch_size=64)

Epoch 1/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 197s 387ms/step - accuracy: 0.7413 - loss: 0.5036 - val_accuracy: 0.8754 - val_loss: 0.3091
Epoch 2/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 199s 381ms/step - accuracy: 0.8825 - loss: 0.2924 - val_accuracy: 0.8808 - val_loss: 0.2922
Epoch 3/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 190s 379ms/step - accuracy: 0.9004 - loss: 0.2529 - val_accuracy: 0.8786 - val_loss: 0.2955
Epoch 4/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 187s 375ms/step - accuracy: 0.8993 - loss: 0.2550 - val_accuracy: 0.8790 - val_loss: 0.2998
Epoch 5/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 377ms/step - accuracy: 0.9256 - loss: 0.1960 - val_accuracy: 0.8780 - val_loss: 0.3171
Epoch 6/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 377ms/step - accuracy: 0.9317 - loss: 0.1811 - val_accuracy: 0.8764 - val_loss: 0.3179
Epoch 7/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 378ms/step - accuracy: 0.9433 - loss: 0.1495 - val_accuracy: 0.8679 - val_loss: 0.3448
Epoch 8/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 377ms/step - accuracy: 0.9525 - loss: 0

In [ ]:
model.evaluate(test_sequences,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 110ms/step - accuracy: 0.8676 - loss: 0.4020


[0.40222087502479553, 0.8687999844551086]

In [ ]:
test_rev = pad_sequences(tokenizer.texts_to_sequences(test_review['Review']),maxlen=200)
model.evaluate(test_rev,test_review['rating'])

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 115ms/step - accuracy: 0.8924 - loss: 0.3082


[0.3006880581378937, 0.895799994468689]

In [ ]:
text = 'this is cinema. such movie make me fall in love with cinema all over again. what a msterpiece it is. from acting to background score, everything is perfect.'

def sentiment(text):
  text = cleaning(text)
  input = pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=200)
  pred = model.predict(input)
  prediction = 'positive' if pred[0][0]>0.5 else 'negative'
  return prediction

sentiment(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step


'positive'

In [ ]:
model.save('review_sentiment3.keras')             # this is the model we using <------

In [ ]:
import pickle

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

### Approach 3 - Word Embedding using Word2Vec

In [ ]:
from nltk.tokenize import sent_tokenize

import gensim
from gensim.models import word2vec,KeyedVectors
from gensim.utils import simple_preprocess

In [ ]:
kaggle_review = pd.read_csv('/content/Reviews.csv')

lb = LabelBinarizer()
kaggle_review['rating'] = lb.fit_transform(kaggle_review['sentiment'])
kaggle_review.drop(columns=['sentiment'],inplace=True)
kaggle_review.rename(columns={'review':'Review'},inplace=True)

In [ ]:
review_texts = kaggle_review['Review'].tolist()
reviews = [text.split() for text in review_texts]

In [ ]:
model = gensim.models.Word2Vec(window=10,min_count=2,vector_size=50)
model.build_vocab(reviews)

In [ ]:
model.train(kaggle_review['Review'],total_examples=model.corpus_count,epochs=model.epochs)

(95896935, 203331885)

In [ ]:
len(model.wv.index_to_key)

61920

In [ ]:
review_token = []
word_to_index = {word: idx+1 for idx, word in enumerate(model.wv.index_to_key)}

embedding_matrix = np.zeros((62072, 50))

for word, idx in word_to_index.items():
    if word in model.wv:
        embedding_matrix[idx] = model.wv[word]

In [ ]:
for rev in kaggle_review['Review']:
  review_token.append([word_to_index[word] for word in rev.split() if word in word_to_index])

In [ ]:
revise = pad_sequences(review_token,maxlen=200)

In [ ]:
Y = kaggle_review['rating'].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(revise,Y,test_size=0.2,random_state=41)
x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape

((40000, 200), (10000, 200), (40000,), (10000,))

In [ ]:
x_train_embedded = np.array([[embedding_matrix[idx] for idx in review] for review in x_train])

In [ ]:
x_test_embedded = np.array([[embedding_matrix[idx] for idx in review] for review in x_test])

In [ ]:
x_train_embedded.shape ,x_test_embedded.shape ,y_train.shape ,y_test.shape

((40000, 200, 50), (10000, 200, 50), (40000,), (10000,))

In [ ]:
model = Sequential()
model.add(Input(shape=(200, 50)))
model.add(LSTM(256,dropout=0.2,recurrent_dropout=0.3,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))

# model = Sequential([
#     Input(shape=(200, 50)),
#     Bidirectional(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),  # First BiLSTM
#     BatchNormalization(),  # Normalize activations
#     Dense(64, activation='relu'),  # Fully connected layer
#     Dropout(0.4),  # Reduce overfitting
#     Dense(1, activation='sigmoid')  # Output for binary classification
# ])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 256)                 │         314,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 314,625 (1.20 MB)

 Trainable params: 314,625 (1.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [ ]:
model.fit(x_train_embedded,y_train,epochs=5,validation_data=[x_test_embedded,y_test],batch_size=64)